#MIT 6.036 Spring 2019: Homework 10#

This colab notebook provides code and a framework for questions 2, 3, and 4 from [homework 10](https://openlearninglibrary.mit.edu/courses/course-v1:MITx+6.036+1T2019/courseware/Week10/week10_homework/).  You can work out your solutions here, then submit your results back on the homework page when ready.

## <section>**Setup**</section>

First, download the code distribution for this homework that contains test cases and helper functions.

Run the next code block to download and import the code for this lab.

In [3]:
#!rm -rf code_for_hw10* __MACOSX data .DS_Store

#!wget --quiet https://introml_oll.odl.mit.edu/cat-soop/_static/6.036/homework/hw10/code_for_hw10.zip
#!unzip code_for_hw10.zip
#!mv code_for_hw10/* .



Archive:  code_for_hw10.zip
   creating: code_for_hw10/
  inflating: code_for_hw10/code_for_hw10.py  
  inflating: code_for_hw10/dist.py   
  inflating: code_for_hw10/mdp10.py  
  inflating: code_for_hw10/util.py   


In [5]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
'''import zipfile
import os

# Define the path to your ZIP file and extraction folder, new files that have TF and Keras
zip_file_path = '/content/drive/My Drive/code_for_hw10.zip'
extract_folder = '/content/drive/My Drive/Colab Notebooks/'

# Create the extraction folder if it doesn't exist
os.makedirs(extract_folder, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print("Extraction completed.")'''


Extraction completed.


In [11]:
!rm -rf code_for_hw10*

In [ ]:
#Make new code_for_hw10 with TF based Keras imports and replace that  before importing to  HW 10 code below
#manually Wget files and unzip locally and upload to related drive root folder so that no above issues are seen

In [15]:
import pdb
import numpy as np
import matplotlib.pyplot as plt
import dist
import util
import pickle
#import pdb
import random
import numpy as np
from dist import uniform_dist, delta_dist, mixture_dist
from util import argmax_with_val, argmax
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

class MDP:
    # Needs the following attributes:
    # states: list or set of states
    # actions: list or set of actions
    # discount_factor: real, greater than 0, less than or equal to 1
    # start: optional instance of DDist, specifying initial state dist
    #    if it's unspecified, we'll use a uniform over states
    # These are functions:
    # transition_model: function from (state, action) into DDist over next state
    # reward_fn: function from (state, action) to real-valued reward

    def __init__(self, states, actions, transition_model, reward_fn,
                     discount_factor = 1.0, start_dist = None):
        self.states = states
        self.actions = actions
        self.transition_model = transition_model
        self.reward_fn = reward_fn
        self.discount_factor = discount_factor
        self.start = start_dist if start_dist else uniform_dist(states)

    # Given a state, return True if the state should be considered to
    # be terminal.  You can think of a terminal state as generating an
    # infinite sequence of zero reward.
    def terminal(self, s):
        return False

    # Randomly choose a state from the initial state distribution
    def init_state(self):
        return self.start.draw()

    # Simulate a transition from state s, given action a.  Return
    # reward for (s,a) and new state, drawn from transition.  If a
    # terminal state is encountered, sample next state from initial
    # state distribution
    def sim_transition(self, s, a):
        return (self.reward_fn(s, a),
                self.init_state() if self.terminal(s) else
                    self.transition_model(s, a).draw())

    def state2vec(self, s):
        '''
        Return one-hot encoding of state s; used in neural network agent implementations
        '''
        v = np.zeros((1, len(self.states)))
        v[0,self.states.index(s)] = 1.
        return v

# Perform value iteration on an MDP, also given an instance of a q
# function.  Terminate when the max-norm distance between two
# successive value function estimates is less than eps.
# interactive_fn is an optional function that takes the q function as
# argument; if it is not None, it will be called once per iteration,
# for visuzalization

# The q function is typically an instance of TabularQ, implemented as a
# dictionary mapping (s, a) pairs into Q values This must be
# initialized before interactive_fn is called the first time.

def value_iteration(mdp, q, eps = 0.01, max_iters = 1000):
    # Your code here (COPY FROM HW9)
    raise NotImplementedError('value_iteration')

# Given a state, return the value of that state, with respect to the
# current definition of the q function
def value(q, s):
    # Your code here (COPY FROM HW9)
    raise NotImplementedError('value')

# Given a state, return the action that is greedy with reespect to the
# current definition of the q function
def greedy(q, s):
    # Your code here (COPY FROM HW9)
    raise NotImplementedError('greedy')

def epsilon_greedy(q, s, eps = 0.5):
    if random.random() < eps:  # True with prob eps, random action
        # Your code here (COPY FROM HW9)
        raise NotImplementedError('epsilon_greedy')
    else:
        # Your code here (COPY FROM HW9)
        raise NotImplementedError('epsilon_greedy')

class TabularQ:
    def __init__(self, states, actions):
        self.actions = actions
        self.states = states
        self.q = dict([((s, a), 0.0) for s in states for a in actions])
    def copy(self):
        q_copy = TabularQ(self.states, self.actions)
        q_copy.q.update(self.q)
        return q_copy
    def set(self, s, a, v):
        self.q[(s,a)] = v
    def get(self, s, a):
        return self.q[(s,a)]
    def update(self, data, lr):
        # Your code here
        raise NotImplementedError('TabularQ.update')

def Q_learn(mdp, q, lr=.1, iters=100, eps = 0.5, interactive_fn=None):
    # Your code here
    raise NotImplementedError('Q_learn')
    for i in range(iters):
        # include this line in the iteration, where i is the iteration number
        if interactive_fn: interactive_fn(q, i)
    pass

# Simulate an episode (sequence of transitions) of at most
# episode_length, using policy function to select actions.  If we find
# a terminal state, end the episode.  Return accumulated reward a list
# of (s, a, r, s') where s' is None for transition from terminal state.
# Also return an animation if draw=True.
def sim_episode(mdp, episode_length, policy, draw=False):
    episode = []
    reward = 0
    s = mdp.init_state()
    all_states = [s]
    for i in range(int(episode_length)):
        a = policy(s)
        (r, s_prime) = mdp.sim_transition(s, a)
        reward += r
        if mdp.terminal(s):
            episode.append((s, a, r, None))
            break
        episode.append((s, a, r, s_prime))
        if draw:
            mdp.draw_state(s)
        s = s_prime
        all_states.append(s)
    animation = animate(all_states, mdp.n, episode_length) if draw else None
    return reward, episode, animation

# Create a matplotlib animation from all states of the MDP that
# can be played both in colab and in local versions.
def animate(states, n, ep_length):
    try:
        from matplotlib import animation, rc
        import matplotlib.pyplot as plt
        from google.colab import widgets

        plt.ion()
        plt.figure(facecolor="white")
        fig, ax = plt.subplots()
        plt.close()

        def animate(i):
            if states[i % len(states)] == None or states[i % len(states)] == 'over':
                return
            ((br, bc), (brv, bcv), pp, pv) = states[i % len(states)]
            im = np.zeros((n, n+1))
            im[br, bc] = -1
            im[pp, n] = 1
            ax.cla()
            ims = ax.imshow(im, interpolation = 'none',
                        cmap = 'viridis',
                        extent = [-0.5, n+0.5,
                                    -0.5, n-0.5],
                        animated = True)
            ims.set_clim(-1, 1)
        rc('animation', html='jshtml')
        anim = animation.FuncAnimation(fig, animate, frames=ep_length, interval=100)
        return anim
    except:
        # we are not in colab, so the typical animation should work
        return None

# Return average reward for n_episodes of length episode_length
# while following policy (a function of state) to choose actions.
def evaluate(mdp, n_episodes, episode_length, policy):
    score = 0
    length = 0
    for i in range(n_episodes):
        # Accumulate the episode rewards
        r, e, _ = sim_episode(mdp, episode_length, policy)
        score += r
        length += len(e)
        # print('    ', r, len(e))
    return score/n_episodes, length/n_episodes

def Q_learn_batch(mdp, q, lr=.1, iters=100, eps=0.5,
                  episode_length=10, n_episodes=2,
                  interactive_fn=None):
    # Your code here
    raise NotImplementedError('Q_learn_batch')
    for i in range(iters):
        # include this line in the iteration, where i is the iteration number
        if interactive_fn: interactive_fn(q, i)
    pass

def make_nn(state_dim, num_hidden_layers, num_units):
    model = Sequential()
    model.add(Dense(num_units, input_dim = state_dim, activation='relu'))
    for i in range(num_hidden_layers-1):
        model.add(Dense(num_units, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer=Adam())
    return model

class NNQ:
    def __init__(self, states, actions, state2vec, num_layers, num_units, epochs=1):
        self.actions = actions
        self.states = states
        self.state2vec = state2vec
        self.epochs = epochs
        self.models = None              # Your code here
        if self.models is None: raise NotImplementedError('NNQ.models')
    def get(self, s, a):
        # Your code here
        raise NotImplementedError('NNQ.get')
    def update(self, data, lr):
        # Your code here
        raise NotImplementedError('NNQ.update')


class No_Exit(MDP):
    # Like breakout or pong, but one player, no walls to break out, no
    # way to win You can move paddle vertically up or down or stay
    actions = (+1, 0, -1)

    def __init__(self, field_size, ball_speed = 1, random_start = True):
        # image space is n by n
        self.q = None
        self.n = field_size
        h = self.n * ball_speed
        self.discount_factor = (h - 1.0) / h
        self.ball_speed = ball_speed
        # state space is: ball position and velocity, paddle position
        # and velocity
        # - ball position is n by n
        # - ball velocity is one of (-1, -1), (-1, 1), (0, -1), (0, 1),
        #                          (1, -1), (1, 1)
        # - paddle position is n; this is location of bottom of paddle,
        #    can stick "up" out of the screen
        # - paddle velocity is one of 1, 0, -1
        self.states = [((br, bc), (brv, bcv), pp, pv) for \
                         br in range(self.n) for
                         bc in range(self.n) for
                         brv in (-1, 0, 1) for
                         bcv in (-1, 1) for
                         pp in range(self.n) for
                         pv in (-1, 0, 1)]
        self.states.append('over')
        self.start = dist.uniform_dist([((br, 0), (0, 1), 0, 0) \
                                        for br in range(self.n)]) \
                if random_start else  \
                dist.delta_dist(((int(self.n/2), 0), (0, 1), 0, 0))

    ax = None
    ims = None
    # Updating values in google colab
    try:
        from google.colab import widgets
        IS_COLAB = True
        grid = widgets.Grid(1, 10, header_row=False, header_column=False)
        parity = 0
    except:
        IS_COLAB = False
        grid = None

    def draw_state(self, state = None, pause = False):
        def _update(self, state, pause):
            if self.ax is None or self.IS_COLAB:
                plt.ion()
                plt.figure(facecolor="white")
                self.ax = plt.subplot()

            if state is None: state = self.state
            ((br, bc), (brv, bcv), pp, pv) = state
            im = np.zeros((self.n, self.n+1))
            im[br, bc] = -1
            im[pp, self.n] = 1
            self.ax.cla()
            self.ims = self.ax.imshow(im, interpolation = 'none',
                                    cmap = 'viridis',
                                    extent = [-0.5, self.n+0.5,
                                                -0.5, self.n-0.5],
                                    animated = True)
            self.ims.set_clim(-1, 1)
            plt.pause(0.0001)
            if pause: input('go?')
            else: plt.pause(0.1 if self.IS_COLAB else 0.01)

        if self.IS_COLAB:
            with self.grid.output_to(0, (self.parity % 10)):
                # _update(self, state, pause)
                self.grid.clear_cell(0, (self.parity + 1) % 10)
                self.parity =  (self.parity + 9) % 10
        else:
            _update(self, state, pause)

    def state2vec(self, s):
        if s == 'over':
            return np.array([[0, 0, 0, 0, 0, 0, 1]])
        ((br, bc), (brv, bcv), pp, pv) = s
        return np.array([[br, bc, brv, bcv, pp, pv, 0]])

    def terminal(self, state):
        return state == 'over'

    def reward_fn(self, s, a):
        return 0 if s == 'over' else 1

    def transition_model(self, s, a, p = 0.4):
        # Only randomness is in brv and brc after a bounce
        # 1- prob of negating nominal velocity
        if s == 'over':
            return dist.delta_dist('over')
        # Current state
        ((br, bc), (brv, bcv), pp, pv) = s
        # Nominal next ball state
        new_br = br + self.ball_speed*brv; new_brv = brv
        new_bc = bc + self.ball_speed*bcv; new_bcv = bcv
        # nominal paddle state, a is action (-1, 0, 1)
        new_pp = max(0, min(self.n-1, pp + a))
        new_pv = a
        new_s = None
        hit_r = hit_c = False
        # bottom, top contacts
        if new_br < 0:
            new_br = 0; new_brv = 1; hit_r = True
        elif new_br >= self.n:
            new_br = self.n - 1; new_brv = -1; hit_r = True
        # back, front contacts
        if new_bc < 0:                  # back bounce
            new_bc = 0; new_bcv = 1; hit_c = True
        elif new_bc >= self.n:
            if self.paddle_hit(pp, new_pp, br, bc, new_br, new_bc):
                new_bc = self.n-1; new_bcv = -1; hit_c = True
            else:
                return dist.delta_dist('over')

        new_s = ((new_br, new_bc), (new_brv, new_bcv), new_pp, new_pv)
        if ((not hit_c) and (not hit_r)):
            return dist.delta_dist(new_s)
        elif hit_c:                     # also hit_c and hit_r
            if abs(new_brv) > 0:
                return dist.DDist({new_s: p,
                                   ((new_br, new_bc), (-new_brv, new_bcv), new_pp, new_pv) : 1-p})
            else:
                return dist.DDist({new_s: p,
                                   ((new_br, new_bc), (-1, new_bcv), new_pp, new_pv) : 0.5*(1-p),
                                   ((new_br, new_bc), (1, new_bcv), new_pp, new_pv) : 0.5*(1-p)})
        elif hit_r:
            return dist.DDist({new_s: p,
                               ((new_br, new_bc), (new_brv, -new_bcv), new_pp, new_pv) : 1-p})


    def paddle_hit(self, pp, new_pp, br, bc, new_br, new_bc):
        # Being generous to paddle, any overlap in row
        prset = set(range(pp, pp+2)).union(set(range(new_pp, new_pp+2)))
        brset = set([br, br+1, new_br, new_br+1])
        return len(prset.intersection(brset)) >= 2

##############################
# Display
##############################
def tidy_plot(xmin, xmax, ymin, ymax, center = False, title = None,
                 xlabel = None, ylabel = None):
    # plt.ion()
    plt.figure(facecolor="white")
    ax = plt.subplot()
    if center:
        ax.spines['left'].set_position('zero')
        ax.spines['right'].set_color('none')
        ax.spines['bottom'].set_position('zero')
        ax.spines['top'].set_color('none')
        ax.spines['left'].set_smart_bounds(True)
        ax.spines['bottom'].set_smart_bounds(True)
        ax.xaxis.set_ticks_position('bottom')
        ax.yaxis.set_ticks_position('left')
    else:
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.get_xaxis().tick_bottom()
        ax.get_yaxis().tick_left()
    eps = .05
    plt.xlim(xmin-eps, xmax+eps)
    plt.ylim(ymin-eps, ymax+eps)
    if title: ax.set_title(title)
    if xlabel: ax.set_xlabel(xlabel)
    if ylabel: ax.set_ylabel(ylabel)
    return ax

def plot_points(x, y, ax = None, clear = False,
                  xmin = None, xmax = None, ymin = None, ymax = None,
                  style = 'or-'):

    if ax is None:
        if xmin == None: xmin = np.min(x) - 0.5
        if xmax == None: xmax = np.max(x) + 0.5
        if ymin == None: ymin = np.min(y) - 0.5
        if ymax == None: ymax = np.max(y) + 0.5
        ax = tidy_plot(xmin, xmax, ymin, ymax)

        x_range = xmax - xmin; y_range = ymax - ymin
        if .1 < x_range / y_range < 10:
            plt.axis('equal')
        xlim, ylim = ax.get_xlim(), ax.get_ylim()
    elif clear:
        xlim, ylim = ax.get_xlim(), ax.get_ylim()
        ax.clear()
    else:
        xlim, ylim = ax.get_xlim(), ax.get_ylim()
    ax.plot(x, y, style, markeredgewidth=0.0)
    # Seems to occasionally mess up the limits
    # ax.set_xlim(xlim); ax.set_ylim(ylim)
    ax.grid(True, which='both')
    plt.show()
    return ax

import functools
def toHex(s):
    lst = []
    for ch in s:
        hv = hex(ord(ch)).replace('0x', '')
        if len(hv) == 1:
            hv = '0'+hv
        lst.append(hv)

    return functools.reduce(lambda x,y:x+y, lst)

##############################

def test_learn_play(d = 6, num_layers = 2, num_units = 100,
                    eps = 0.5, iters = 10000, draw=False,
                    tabular = True, batch=False, batch_epochs=10,
                    num_episodes = 10, episode_length = 100):
    iters_per_value = 1 if iters <= 10 else int(iters / 10.0)
    scores = []
    def interact(q, iter=0):
        if iter % iters_per_value == 0:
            scores.append((iter, evaluate(game, num_episodes, episode_length,
                                          lambda s: greedy(q, s))[0]))
            print('score', scores[-1], flush=True)
    game = No_Exit(d)
    if tabular:
        q = TabularQ(game.states, game.actions)
    else:
        q = NNQ(game.states, game.actions, game.state2vec, num_layers, num_units,
                epochs=batch_epochs if batch else 1)
    if batch:
        qf = Q_learn_batch(game, q, iters=iters, episode_length = 100, n_episodes=10,
                           interactive_fn=interact)
    else:
        qf = Q_learn(game, q, iters=iters, interactive_fn=interact)
    if scores:
        print('String to upload (incude quotes): "%s"'%toHex(pickle.dumps([tabular, batch, scores], 0).decode()))
        # Plot learning curve
        plot_points(np.array([s[0] for s in scores]),
                    np.array([s[1] for s in scores]))
    for i in range(num_episodes):
        reward, _, animation = sim_episode(game, (episode_length if d > 5 else episode_length/2),
                                lambda s: greedy(qf, s), draw=draw)
        print('Reward', reward)
    return animation


def test_solve_play(d = 6, draw=False,
                    num_episodes = 10, episode_length = 100):
    game = No_Exit(d)
    qf = value_iteration(game , TabularQ(game.states, game.actions))
    for i in range(num_episodes):
        reward, _, animation = sim_episode(game, (episode_length if d > 5 else episode_length/2),
                                lambda s: greedy(qf, s), draw=draw)
        print('Reward', reward)
    return animation

##########   Test cases

# Value Iteration
# test_solve_play()
# Tabular Q-learn
# test_learn_play(iters=100000, tabular=True, batch=False)
# Tabular Batch Q-learn
# test_learn_play(iters=10, tabular=True, batch=True) # Check: why do we want fewer iterations here?
# NN Q-learn
# test_learn_play(iters=100000, tabular=False, batch=False)
# NN Batch Q-learn (Fitted Q-learn)
# test_learn_play(iters=10, tabular=False, batch=True)


,,,,,,,,,


In [16]:

#import code_for_hw10 as code_for_hw10
#import mdp10 as mdp

import numpy as np
import math as m
import random

import pdb
from dist import uniform_dist, delta_dist, mixture_dist, DDist
from util import argmax_with_val, argmax
#from keras.models import Sequential
#from keras.layers.core import Dense
#from keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

import importlib

# 2) Implement Q-Learning

We'll work up to implementing the Q-learning algorithm by extending our code from HW9. In the next block, please copy and paste your implementations of the following functions from HW9.

In [21]:
def value_iteration(mdp, q, eps = 0.01, max_iters = 1000):
    # Your code here (COPY FROM HW9)
    for i in range(max_iters):
        max_value = 0
        max_a, max_s = None,None
        new_q = q.copy()
        for pairs in q.q:
            s,a = pairs[0], pairs[1]
            v = mdp.reward_fn(s,a)
            v += mdp.discount_factor * mdp.transition_model(s,a).expectation(lambda s: value(q,s))
            new_q.set(s,a,v)
            if v>max_value:
                max_value = v
                max_a,max_s = a,s
        if new_q.q[(max_s,max_a)] - q.q[(max_s,max_a)]<eps:
            return new_q
        q = new_q.copy()

    raise NotImplementedError('value_iteration')

def value(q, s):
    # Your code here (COPY FROM HW9)
    q_star = None
    for a in q.actions:
        if q_star is None:
            q_star = q.q[(s,a)]
        elif q.q[(s,a)] > q_star:
            q_star = q.q[(s,a)]
    return q_star

    raise NotImplementedError('value')

def greedy(q, s):
    # Your code here (COPY FROM HW9)
    return argmax(q.actions, lambda a: q.get(s, a))

    raise NotImplementedError('greedy')

def epsilon_greedy(q, s, eps = 0.5):
    if random.random() < eps:  # True with prob eps, random action
        # Your code here (COPY FROM HW9)
        return uniform_dist(q.actions).draw()
        raise NotImplementedError('epsilon_greedy')
    else:
        # Your code here (COPY FROM HW9)
        return action
        raise NotImplementedError('epsilon_greedy')

Run the next code block to make sure what you need from HW9 is working.

In [22]:
MDP.value = value
MDP.greedy = greedy
MDP.epsilon_greedy = epsilon_greedy
MDP.value_iteration = value_iteration

#importlib.reload(code_for_hw10)

# Test: Value Iteration
#code_for_hw10.test_solve_play()

test_solve_play()
# Expected output:
# '''
# Reward 100
# Reward 100
# Reward 100
# Reward 100
# Reward 100
# Reward 100
# Reward 100
# Reward 100
# Reward 100
# Reward 100
# '''

Reward 100
Reward 100
Reward 100
Reward 100
Reward 100
Reward 100
Reward 100
Reward 100
Reward 100
Reward 100


## 2.1) Q update

First, we'll extend our implementation of the TabularQ class in HW 9 (Problem 5) to incorporate the crucial operation of Q-learning, which is to update the Q value for a given `(s, a)` entry and move it part of the way towards a "target" value *t*.

> *Q(s,a) ← (1−α)Q(s,a) + αt*

Note that this can also be written as:

> *Q(s,a)← Q(s,a)+α(t−Q(s,a))*

That is, move a small (*α*) step towards t.

Let's define a new method for `TabularQ` that implements this, in a batched form. We will be given a list of `(s, a, t)` triples and have to do all the updates. Note that update is a method of the `TabularQ` class, so you can access the other methods and attributes.

* `data` is a list of `(s, a, t)` tuples.
* `lr` is a learning rate (*α* above)
* We will have to update `self.q[(s,a)]` for all of the data.


In [23]:
class TabularQ:
    def __init__(self, states, actions):
        self.actions = actions
        self.states = states
        self.q = dict([((s, a), 0.0) for s in states for a in actions])
    def copy(self):
        q_copy = TabularQ(self.states, self.actions)
        q_copy.q.update(self.q)
        return q_copy
    def set(self, s, a, v):
        self.q[(s,a)] = v
    def get(self, s, a):
        return self.q[(s,a)]
    def update(self, data, lr):
        # Your code here
        pass
        for d in data:
            s, a, t = d[0], d[1], d[2]
            self.set(s,a, self.get(s,a)+lr*(t-self.get(s,a)))

## 2.2) Q_learn

Complete the definition of the `Q_learn` function. It should update the entries in the `q` function, `(s, a)`, towards their estimated Q values. It should terminate after `iters` iterations and use learning rate `lr`. Use the `q.update` method, which you just wrote, to update the Q values.

You will need to both simulate the agent's trajectory through the space as well as perform the updates to the Q function estimates. In this version, you should update the Q values after every transition, using a single `(s, a, t)` tuple. **The following methods and functions have already been defined for you.**


* To start a new simulation, call `mdp.init_state()`. That will draw a state from the MDP's initial state distribution.
* You can use the functions that we defined in HW 9: `epsilon_greedy` for action selection (epsilon_greedy takes `(q, s, eps = 0.5)` as input and returns an action) `value` takes `(q, s)` and returns the max Q value for a state.
* To take a step in the simulation, starting in a given state, `s`, using action a, call `mdp.sim_transition(s,a)`. It will return a pair `(r, s_prime)` denoting the reward received by the agent at that step and the next state.
* Be careful in treating terminal states. Recall that at a terminal state, there may be an immediate reward but the future expected value will be zero.
* Return `q` so that the Tutor can test it.

In [26]:
def Q_learn(mdp, q, lr=.1, iters=100, eps = 0.5, interactive_fn=None):
    # Your code here
    s = mdp.init_state()
    for i in range(iters):
        # Your code here
        # include this line in the iteration, where i is the iteration number
        if interactive_fn: interactive_fn(q, i)
        pass
# select action
        a = epsilon_greedy(q, s, eps=eps)
        # execute selected action
        r, s_prime = mdp.sim_transition(s,a)
        if mdp.terminal(s):
            target = r
        else:
            # find max q in for next state and action
            max_q = value(q, s_prime)
            target = r + mdp.discount_factor * max_q
        data = [(s,a,target)]
        q.update(data, lr)
        s = s_prime
    return q

Run the next code blocks to test your implementation of `Q_learn`.

In [27]:
MDP.TabularQ = TabularQ
MDP.Q_learn = Q_learn
#importlib.reload(code_for_hw10)

# Test: Tabular Q-learn
#code_for_hw10.test_learn_play(iters=100000, tabular=True, batch=False)
test_learn_play(iters=100000, tabular=True, batch=False)

# Due to newer TF version, Keras and import issues, please copy required functions from external folders to this file and rename the calling methods to resolve issues like below

score (0, 9.9)


NameError: name 'action' is not defined

In [ ]:
def tinyTerminal(s):
    return s==4
def tinyR(s, a):
    if s == 1: return 1
    elif s == 3: return 2
    else: return 0
def tinyTrans(s, a):
    if s == 0:
        if a == 'a':
            return DDist({1 : 0.9, 2 : 0.1})
        else:
            return DDist({1 : 0.1, 2 : 0.9})
    elif s == 1:
        return DDist({1 : 0.1, 0 : 0.9})
    elif s == 2:
        return DDist({2 : 0.1, 3 : 0.9})
    elif s == 3:
        return DDist({3 : 0.1, 0 : 0.5, 4 : 0.4})
    elif s == 4:
        return DDist({4 : 1.0})

def testQ():
    tiny = mdp.MDP([0, 1, 2, 3, 4], ['a', 'b'], tinyTrans, tinyR, 0.9)
    tiny.terminal = tinyTerminal
    q = TabularQ(tiny.states, tiny.actions)
    qf = Q_learn(tiny, q)
    ret = list(qf.q.items())
    expected = [((0, 'a'), 0.6649739221724159), ((0, 'b'), 0.1712369526453748),
                ((1, 'a'), 0.7732751316011999), ((1, 'b'), 1.2034912054227331),
                ((2, 'a'), 0.37197205380133874), ((2, 'b'), 0.45929063274463033),
                ((3, 'a'), 1.5156163024818292), ((3, 'b'), 0.8776852768653631),
                ((4, 'a'), 0.0), ((4, 'b'), 0.0)]
    ok = True
    for (s,a), v in expected:
      qv = qf.get(s,a)
      if abs(qv-v) > 1.0e-5:
        print("Oops!  For (s=%s, a=%s) expected %s, but got %s" % (s, a, v, qv))
        ok = False
    if ok:
      print("Tests passed!")

random.seed(0)
testQ()

## 2.3) Batch Q_learn

Assume your previous update method has been defined.

In the standard Q-learning algorithm, we make one epsilon-greedy transition based on the current Q estimate and then update the Q values. You can think of this as being like stochastic gradient descent. We can also define a version that is more like batch gradient descent, where we generate one or more "episodes" (sequences of transitions) using the current Q values and then update the Q values based on all the observed results. We can also keep around old transitions and use them (all or a random subset) in the update as well. **Note that as our Q value estimate evolves, the target Q value computed from a previously observed transition can change.**

Implement this version of batch Q-learning that (a) generates some specifed number of episodes of a given length (see `sim_episode` below), (b) adds these to the experiences we have seen previously, and (c) updates the Q estimates based on **all the experience so far**. Return `q` so that the Tutor can test it.

In [ ]:
# evaluate this cell so you can use the definition in your code below

def sim_episode(mdp, episode_length, policy, draw=False):
    '''
    Simulate an episode (sequence of transitions) of at most
    episode_length, using policy function to select actions.  If we find
    a terminal state, end the episode.  Return accumulated reward a list
    of (s, a, r, s') where s' is None for transition from terminal state.
    Also return an animation if draw=True, or None if draw=False
    '''
    episode = []
    reward = 0
    s = mdp.init_state()
    all_states = [s]
    for i in range(episode_length):
        a = policy(s)
        (r, s_prime) = mdp.sim_transition(s, a)
        reward += r
        if mdp.terminal(s):
            episode.append((s, a, r, None))
            break
        episode.append((s, a, r, s_prime))
        if draw:
            mdp.draw_state(s)
        s = s_prime
        all_states.append(s)
    animation = animate(all_states, mdp.n, episode_length) if draw else None
    return reward, episode, animation

**Clarifications**

* There should be a SINGLE call to `q.update` per iteration, not per episode or per experience. Just one call per iteration, with a lot of data.
* Let's understand the distinction between experiences `(s, a, r, s')` and Q targets `(s, a, t)`. Note that experiences don't depend on the current estimated Q values (only on the environment we are acting in), but the "t" in the Q targets depends on the current Q values. So, it makes sense to store experiences across iterations, but not to store Q targets, since the Q targets change when we update our Q values. Thus, you want to continuously aggregate the experience and then, in each iteration, re-compute the Q targets under the current estimated Q values, then do the update with all of these Q targets. Here's pseudocode:


```
all_experiences = []
Loop over n_iterations:
    Loop over n_episodes:
        Generate an episode of length episode_length, append this experience to all_experiences
    all_q_targets = []
    Loop over all_experiences:
        Append Q target from one experience to all_q_targets
        Remember to handle terminal states (where s' = None)
    q.update(all_q_targets, lr)
return q
```



In [28]:
def Q_learn_batch(mdp, q, lr=.1, iters=100, eps=0.5,
                  episode_length=10, n_episodes=2,
                  interactive_fn=None):
    # Your code here
    all_experience = []
    for i in range(iters):
        # include this line in the iteration, where i is the iteration number
        if interactive_fn: interactive_fn(q, i)
        for j in range(n_episodes):
            reward, episode, animation = sim_episode(mdp, episode_length, lambda s: epsilon_greedy(q,s,eps=eps), draw=False)
            for e in episode:
                all_experience.append(e)

        # calculate target using [(s, a, r, s')] pairs in all_experience
        all_q_targets = []
        for e in all_experience:
            if e[3] is not None:
                t = e[2]+mdp.discount_factor * value(q, e[3])
            else:
                t = e[2]
            all_q_targets.append((e[0], e[1], t))

        q.update(all_q_targets, lr)
    return q

Run the next code blocks to test your implementation of `Q_learn_batch`.

In [ ]:
MDP.Q_learn_batch = Q_learn_batch
#importlib.reload(code_for_hw10)

# Test: Tabular Batch Q-learn
#code_for_hw10.test_learn_play(iters=10, tabular=True, batch=True) # Check: why do we want fewer iterations here?
test_learn_play(iters=10, tabular=True, batch=True) # Check: why do we want fewer iterations here?

In [ ]:
def testBatchQ():
    tiny = mdp.MDP([0, 1, 2, 3, 4], ['a', 'b'], tinyTrans, tinyR, 0.9)
    tiny.terminal = tinyTerminal
    q = TabularQ(tiny.states, tiny.actions)
    qf = Q_learn_batch(tiny, q)
    ret = list(qf.q.items())
    expected = [((0, 'a'), 4.7566600197286535), ((0, 'b'), 3.993296047838986),
                ((1, 'a'), 5.292467934685342), ((1, 'b'), 5.364014782870985),
                ((2, 'a'), 4.139537149779127), ((2, 'b'), 4.155347555640753),
                ((3, 'a'), 4.076532544818926), ((3, 'b'), 4.551442974149778),
                ((4, 'a'), 0.0), ((4, 'b'), 0.0)]

    ok = True
    for (s,a), v in expected:
      qv = qf.get(s,a)
      if abs(qv-v) > 1.0e-5:
        print("Oops!  For (s=%s, a=%s) expected %s, but got %s" % (s, a, v, qv))
        ok = False
    if ok:
      print("Tests passed!")

      return list(qf.q.items())

random.seed(0)
testBatchQ()

# 3) NN Q: Using neural networks to store the Q function

We would like to operate in large or continuous state and/or action
spaces so it is not possible (or effective) to store the $Q$ values in
a table as we did with the <tt>TabularQ</tt> class; instead, we will
"store" them by training a neural network to do regression for us,
taking $s,a$ as input and generating (an approximation of) $Q^*(s,a)$
as output.

To train the network, we will use <i>squared Bellman error</i> as
the loss function:
$$\left(\left[R(s_t, a_t) + \gamma \max_{a'} Q(s_{t+1}, a'; \theta)\right]
- Q(s_t, a_t;\theta) \right)^2$$
where $\theta$ stands for the current weights in the neural network
and $Q(s, a; \theta)$ stands for the output of the network with
weights $\theta$ when $(s,a)$ is the input.

There are many choices of neural network architecture for storing Q
values.  In this problem, we will:

<ul>

<li> Focus on the case where we have a small set of possible actions,
so make one neural network for each possible action <math>a</math>;

<li> Design that network with two <b>hidden</b> layers with ReLU units
and a single linear output unit (although a deeper network could be
useful); and

<li> Use mean squared error (MSE) as the loss function since, we are
predicting continuous <math>Q</math> values, which is a regression
problem.

</ul>

To use a neural net to store Q values, for a given action, we will
need to have a mapping from states to fixed-length vectors.  We will
assume that the <code>MDP</code> class has a <code>state2vec</code>
method that maps states to vectors.  For the simple discrete-state
MDPs we have seen so far, this simply returns a one-hot representation
of the state.

For reference, this is our implementation of
<code>state2vec</code> (note the shape of its returned array):
<pre>
    def state2vec(self, s):
        '''
        Return one-hot encoding of state s; used in neural network agent implementations
        '''
        v = np.zeros((1, len(self.states)))
        v[0,self.states.index(s)] = 1.
        return v
</pre>

Now, all we need to do is write a new class, called <code>NNQ</code>
to implement neural-network version of Q-function storage; then we can
pass an <code>NNQ</code> instance instead of a <code>TabularQ</code> instance
into <code>Q_learn</code> or <code>Q_learn_batch</code>, and
we will automatically have reinforcement learning with neural
networks!

There are three methods to implement in our <code>NNQ</code>
class. Here are some ideas for how to do that:
<ul>

<li> <code>__init__</code>: Create one neural network for each action,
and store them in <tt>self.models</tt>. Note that <tt>actions</tt> is
a list that may consist of integers or strings or other objects.  As a
reminder, here's how to make a new feed-forward network using Keras:


In [ ]:
# please evaluate this cell so you can use it in your code

def make_nn(state_dim, num_hidden_layers, num_units):
    '''
    state_dim =	(int) number of states
    num_hidden_layers =	(int) number of	fully connected hidden layers
    num_units =	(int) number of	dense relu units to use	in hidden layers
    '''
    model = Sequential()
    model.add(Dense(num_units, input_dim = state_dim, activation='relu'))
    for i in range(num_hidden_layers-1):
        model.add(Dense(num_units, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer=Adam())
    return model

<ul>
<li> <code>get(self, s, a)</code>: Use the neural network you have
stored for action <code>a</code> to predict a Q value for state
<code>s</code>. Feel free to consult documentation on the
<a href="https://keras.io/models/model/#predict">Keras model predict</a>
method.</li>

<li> <code>update(self, data, lr, epochs = 1)</code>: As in
<code>TabularQ</code>, <code>data</code> is a list of <code>(s, a,t)</code>
tuples, where <code>t</code> is a target Q value.  For each
action <code>a</code>, you will need to:

  <ul>
  <li> Construct a training set <code>X, Y</code> of data that is
  relevant to action <code>a</code>, where the input values are states
  (encoded as vectors) and the output values are the target Q values;</li>

  <li> Use the Keras method <code>fit(self, X, Y, epochs=epochs)</code>
  to update the weights in th e associated network.  You can ignore
  the <tt>lr</tt> input parameter, and let Adam in keras manage the
  learning rate.</li>
  </ul>
</li>
</ul>

In [29]:
# Complete the following definition to implement the NNQ class

class NNQ:
    def __init__(self, states, actions, state2vec, num_layers, num_units, epochs=1):
        self.actions = actions
        self.states = states
        self.epochs = epochs
        self.state2vec = state2vec
        state_dim = state2vec(states[0]).shape[1] # a row vector
        self.models = {a:make_nn(state_dim, num_layers, num_units) for a in actions}
    def get(self, s, a):
        return self.models[a].predict(self.state2vec(s))
    def update(self, data, lr):
        for a in self.actions:
            if [s for (s, at, t) in data if a==at]:
                X = np.vstack([self.state2vec(s) for (s, at, t) in data if a==at])
                Y = np.vstack([t for (s, at, t) in data if a==at])
                self.models[a].fit(X, Y, epochs = self.epochs, verbose = False)

Run the next code blocks to test your implementation of `NNQ`:

In [ ]:
mdp.NNQ = NNQ
importlib.reload(code_for_hw10)

# Test: NN Q-learn
code_for_hw10.test_learn_play(iters=100000, tabular=False, batch=False)

In [ ]:
def test_NNQ(data):
    tiny = mdp.MDP([0, 1, 2, 3, 4], ['a', 'b'], tinyTrans, tinyR, 0.9)
    tiny.terminal = tinyTerminal
    q = NNQ(tiny.states, tiny.actions, tiny.state2vec, 2, 10)
    q.update(data, 1)
    ret =  [q.get(s,a) for s in q.states for a in q.actions]
    expect = [np.array([[-0.07211456]]), np.array([[-0.19553234]]),
              np.array([[-0.21926211]]), np.array([[0.01699455]]),
              np.array([[-0.26390356]]), np.array([[0.06374809]]),
              np.array([[0.0340214]]), np.array([[-0.18334733]]),
              np.array([[-0.438375]]), np.array([[-0.13844737]])]
    cnt = 0
    ok = True
    for s in q.states:
      for a in q.actions:
        if not np.all(np.abs(ret[cnt]-expect[cnt]) < 1.0e0):
          print("Oops, for s=%s, a=%s expected %s but got %s" % (s, a, expect[cnt], ret[cnt]))
          ok = False
        cnt += 1
    if ok:
      print("Output looks generally ok")
    return q

test_NNQ([(0,'a',0.3),(1,'a',0.1),(0,'a',0.1),(1,'a',0.5)])

## 3.2) Fitted Q iteration

*Fitted Q iteration (FQ)* suffers less from the correlated experience problem and is generally more stable (and sometimes slower) than NNQ.

FQ initializes the Q networks and an empty data set, then operates in a loop:

1. Use *ϵ*-greedy exploration to generate *k* steps of experience, of the form *(s,a,r,s′)* and add them to the data set.
2. Create one training set for each action *a*:

> 1. Extract all the tuples from your data set that contain action *a*,
> 2. Let the *X* values of your training set be all of the *s* values from your data tuples with action *a* and the *Y* values be the *r + γ max_a' Q(s', a')* values computed for each data tuple, using the Q estimates from the current network.

3. Train the network for action *a* for several epochs until it has done a good job of representing this data.

So, this is basically `Q_learn_batch` using `NNQ` (training with multiple epochs) to implement the Q function.

Run the next code block to test your implementation of `NNQ` with batching.

In [ ]:
# Test: NN Batch Q-learn (Fitted Q-learn)
code_for_hw10.test_learn_play(iters=10, tabular=False, batch=True)

#4) No Exit

Please read the instructions in the [homework](https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week10/week10_homework/) to set up for the game. You may use this space to display the game in Colab.

For each of the learning method and Q model combinations below,
solve the game so that it reliably gets to reward of 100 (that is, the
learned game reliably plays 100 steps without missing the ball,
earning a score of 100).  During learning, you should see a sequence
of lines like: <code>score (5000, 37.5)</code>, which indicates that
after 5000 iterations the average reward over 10 games is 37.5.  We
are checking whether you reach a solution that gets an average reward
100 at least one time. Try playing around with the number of
iterations (an argument to <code>test_learn_play</code>) until you
achieve this point. Note that we will need fewer iterations for
Q_learn_batch, in general (check yourself: why?). After learning, the
code prints a long "upload string" in HEX code.  Enter the upload
strings in the question boxes in the homework MITx site.

In [ ]:
# Value Iteration
code_for_hw10.test_solve_play(draw = True)

In [ ]:
# Tabular Q-learn
code_for_hw10.test_learn_play(draw=True, iters=100000, tabular=True, batch=False)

In [ ]:
# Tabular Batch Q-learn
code_for_hw10.test_learn_play(draw=True, iters=10, tabular=True, batch=True) # Check: why do we want fewer iterations here?

In [ ]:
# NN Q-learn
code_for_hw10.test_learn_play(draw=True, iters=100000, tabular=False, batch=False)

In [ ]:
# NN Batch Q-learn (Fitted Q-learn)
code_for_hw10.test_learn_play(draw=True, iters=10, tabular=False, batch=True)